In [1]:
# Import
import pandas as pd
import yfinance as yf
from pathlib import Path

In [2]:
## SPY
options_df_SPY = pd.DataFrame([])
columns_selected = ['[QUOTE_DATE]', '[UNDERLYING_LAST]', '[EXPIRE_DATE]', '[DTE]', '[C_DELTA]', '[C_VOLUME]',\
                    '[C_BID]', '[C_ASK]', '[STRIKE]', '[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_VOLUME]']
break_val = False

for year_int in range(2010,2100):
    
    year = str(year_int)

    for month_int in range(1,13):

        try:
            month = f'{month_int:02d}'
            df_new = pd.read_csv('text_files_SPY/spy_eod_'+year+month+'.txt',\
                                 sep=', ', engine='python', usecols=columns_selected)
            options_df_SPY = pd.concat([options_df_SPY, df_new], axis=0)
        except Exception:
            break_val = True
            break  

    if break_val == True:
        break

options_df_SPY['[QUOTE_DATE]'] = pd.to_datetime(options_df_SPY['[QUOTE_DATE]'], format='%Y-%m-%d')
options_df_SPY['[EXPIRE_DATE]'] = pd.to_datetime(options_df_SPY['[EXPIRE_DATE]'], format='%Y-%m-%d')

In [3]:
## QQQ
options_df_QQQ = pd.DataFrame([])
columns_selected = ['[QUOTE_DATE]', '[UNDERLYING_LAST]', '[EXPIRE_DATE]', '[DTE]', '[C_DELTA]', '[C_VOLUME]',\
                    '[C_BID]', '[C_ASK]', '[STRIKE]', '[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_VOLUME]']
break_val = False

for year_int in range(2012,2100):
    
    year = str(year_int)

    for month_int in range(1,13):

        try:
            month = f'{month_int:02d}'
            df_new = pd.read_csv('text_files_QQQ/qqq_eod_'+year+month+'.txt',\
                                 sep=', ', engine='python', usecols=columns_selected)
            options_df_QQQ = pd.concat([options_df_QQQ, df_new], axis=0)
        except Exception:
            break_val = True
            break  

    if break_val == True:
        break

options_df_QQQ['[QUOTE_DATE]'] = pd.to_datetime(options_df_QQQ['[QUOTE_DATE]'], format='%Y-%m-%d')
options_df_QQQ['[EXPIRE_DATE]'] = pd.to_datetime(options_df_QQQ['[EXPIRE_DATE]'], format='%Y-%m-%d')

In [4]:
## Obtain min and max dates
# Obtain SPY options dataframe
start_date_SPY = options_df_SPY['[QUOTE_DATE]'].min()
end_date_SPY = options_df_SPY['[QUOTE_DATE]'].max()

# Obtain QQQ options dataframe
start_date_QQQ = options_df_QQQ['[QUOTE_DATE]'].min()
end_date_QQQ = options_df_QQQ['[QUOTE_DATE]'].max()

# Obtain start and end date corresponding to the available options data
start_date = min([start_date_SPY, start_date_QQQ])
end_date = max([end_date_SPY, end_date_QQQ])

In [5]:
## Obtain the risk free rate
# The risk free rate is assumed to be equal to the 10-year US treasury bond yield
rfr_ticker = "^TNX"
rfr_data = yf.download(rfr_ticker, start=start_date, end=end_date, progress=False)

RFR = rfr_data['Adj Close'] * 1/100

# Check if IV is a DataFrame
if isinstance(RFR, pd.DataFrame):
    # Rename the column to 'VIX'
    if RFR.shape[1] == 1:  # Ensure it's a single-column DataFrame
        RFR.columns = ['RFR']
    else:
        raise ValueError("The DataFrame 'IV' should only have one column.")
elif isinstance(RFR, pd.Series):
    # Convert Series to DataFrame and rename the column
    IV = RFR.to_frame(name='RFR')
else:
    raise TypeError("'RFR' must be a pandas DataFrame or Series.")

RFR.index.name = '[QUOTE_DATE]'
# RFR = RFR.rename(columns={'Adj Close': 'RFR'})

# Merge the IV values into options_df_SPY
options_df_SPY = options_df_SPY.join(RFR, how='left')

# Rename the index of IV to match options_df_SPY for merging
RFR.index.name = '[QUOTE_DATE]'

# Ensure the index of IV is datetime if not already
RFR.index = pd.to_datetime(RFR.index)

# SPY
options_df_SPY['[QUOTE_DATE]'] = pd.to_datetime(options_df_SPY['[QUOTE_DATE]'])
options_df_SPY = options_df_SPY.merge(RFR, left_on='[QUOTE_DATE]', right_index=True, how='left', suffixes=('_delete', ''))
if 'RFR_delete' in options_df_SPY.columns:
    del options_df_SPY['RFR_delete']

# QQQ
options_df_QQQ['[QUOTE_DATE]'] = pd.to_datetime(options_df_QQQ['[QUOTE_DATE]'])
options_df_QQQ = options_df_QQQ.merge(RFR, left_on='[QUOTE_DATE]', right_index=True, how='left', suffixes=('_delete', ''))
if 'RFR_delete' in options_df_QQQ.columns:
    del options_df_QQQ['RFR_delete']

In [6]:
try:
    options_df_SPY.to_pickle('SPY_options_data.pkl')
    options_df_QQQ.to_pickle('QQQ_options_data.pkl')
except Exception:
    options_df_SPY.to_pickle(Path('SPY_options_data.pkl'))
    options_df_QQQ.to_pickle(Path('QQQ_options_data.pkl'))

In [7]:
print(options_df_SPY)
print(options_df_QQQ)

      [QUOTE_DATE]  [UNDERLYING_LAST] [EXPIRE_DATE]   [DTE]  [C_DELTA]  \
0       2010-01-04             113.29    2010-01-15    11.0    0.88304   
1       2010-01-04             113.29    2010-01-15    11.0    0.88398   
2       2010-01-04             113.29    2010-01-15    11.0    0.88657   
3       2010-01-04             113.29    2010-01-15    11.0    0.88750   
4       2010-01-04             113.29    2010-01-15    11.0    0.88970   
...            ...                ...           ...     ...        ...   
79388   2023-12-29             475.31    2026-12-18  1085.0    0.07062   
79389   2023-12-29             475.31    2026-12-18  1085.0    0.06917   
79390   2023-12-29             475.31    2026-12-18  1085.0    0.07519   
79391   2023-12-29             475.31    2026-12-18  1085.0    0.05853   
79392   2023-12-29             475.31    2026-12-18  1085.0    0.05847   

       [C_VOLUME]  [C_BID]  [C_ASK]  [STRIKE]  [P_BID]  [P_ASK]  [P_DELTA]  \
0             0.0    58.20    58.